# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [194]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [195]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: @EikiMarcelo

In [196]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @EikiMarcelo

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [197]:
#Produto escolhido:
produto = 'Skol'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [198]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

In [199]:
msgs

['segunda pedindo uma batida de morango com skol beats e leite condensado, só pra dormir feliz mais tarde 🖤🤤🍓',
 'eu não te trouxe flores eu te trouxe brahma, te trouxe skol',
 'meu corpo pede uma skol trincando',
 'rt @neres_jonh: @loveinrio_021\n#lovefestival\n💥 pk class a\n🎤 adamovich \n🎤 mc/dj juninho 22\n🎧 dj diego do karatê\n🎧dj xande cdd\n🎧dj rhuanzada…',
 'rt @diegoviisck: melhor open bar de skol beats de pvh tá voltando com força total e novidades que vcs vão amar.',
 'bebe um litrão de skol ne* https://t.co/huhn9g023o',
 'galera, a sm produções está com uma mega novidade para vcs. para o ano de 2020 começar com o pé direito, terá a melhor festa de todas que já existiu em porto velho, é com open de skol beats no dia 11 de janeiro de 2020. marque o seu amigo aqui e dá um rt. https://t.co/xpjwi4w6zd',
 'rt @lu1zd4vi: tudo certo galera para o maior open de skol beats, vai ficar para a história 😋😋 cool party',
 '@luizaviana13 @rosetylwer disse a que tomou um gole de skol beats e j

Salvando os dados em uma planilha Excel:

In [200]:
len (msgs)

500

In [201]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto), engine = 'openpyxl')

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [202]:
#def de limpar palavras
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
#Excel
skol_read=pd.read_excel("skol.xlsx")

In [203]:
#Limpeza e separação por relevancia
skol_read.Treinamento.apply(cleanup)
skol_read.rename(columns={'Unnamed: 1':'relevante'},inplace=True)
#Relevante 0
skol_relevante_0=skol_read[skol_read.relevante==0]

#Relevante 1
skol_relevante_1=skol_read[skol_read.relevante==1]
print ("Len relevante 0:",len(skol_relevante_0),"",
       "Len relevante 1:",len(skol_relevante_1))

Len relevante 0: 185  Len relevante 1: 115


In [206]:
# Separando as palavras Relevancia 1
" ".join(skol_relevante_1["Treinamento"]).split()
palavras=pd.DataFrame("".join(skol_relevante_1["Treinamento"]).split())
palavras[0].value_counts()
palavras_relevancia_1=palavras[0].value_counts()

In [207]:
# Separando as palavras Relevancia 0
" ".join(skol_relevante_0["Treinamento"]).split()
palavras=pd.DataFrame("".join(skol_relevante_0["Treinamento"]).split())
palavras[0].value_counts()
palavras_relevancia_0=palavras[0].value_counts()
palavras_relevancia_0

skol                                     115
de                                        98
e                                         67
a                                         63
beats                                     41
o                                         38
eu                                        37
da                                        35
que                                       33
pra                                       30
uma                                       29
é                                         27
do                                        27
me                                        23
com                                       22
no                                        20
q                                         18
não                                       18
em                                        17
na                                        16
um                                        16
se                                        16
tu        

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**